In [1]:
import geopandas as gpd
import requests
from zipfile import ZipFile

from utils import (solve_path, solve_dir, check_file_exists,
                delete_existing_files, list_files_recursive)

/home/h-pgy/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
ORIGINAL_DATA_DIR = solve_dir('original_data')
GENERATED_DATA_DIR = solve_dir('generated_data')
GENERATED_DATA_DIR = solve_path('organizacao_territorial', GENERATED_DATA_DIR)
GENERATED_DATA_DIR = solve_dir(GENERATED_DATA_DIR)

## Download ShapeFiles

In [3]:
def geosampa_download_url(arquivo):
    url = (
    'http://download.geosampa.prefeitura.sp.gov.br/'
    'PaginasPublicas/downloadArquivo.aspx?orig=DownloadCamadas'
    f'&arq={arquivo}'
    '&arqTipo=Shapefile'
    )
    
    return url

In [4]:
def zip_fname(fname, folder):
    
    return solve_path(f'{fname}.zip', folder)


In [5]:
def download_shape(fpath, url):
    
    
    with requests.get(url) as r:
        content = r.content
    with open(fpath, 'wb') as f:
        f.write(content)
        
    

In [6]:
def unzip(zip_path, target_dir):
    
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)


In [7]:
def find_shps(tema):
    
    folder_tema = solve_path(tema, GENERATED_DATA_DIR)
    shps=  list_files_recursive(folder_tema, '.shp')
    
    return shps

In [8]:
def download_all_files(de_para_arquivos, folder = GENERATED_DATA_DIR):
    
    for tema, link in de_para_arquivos.items():
        
        shps = find_shps(tema)
        
        if len(shps)<1:
            print(f'Baixando dados para tema {tema}')
            url = geosampa_download_url(link)
            fname = zip_fname(tema, folder)

            download_shape(fname, url)
            tema_folder = solve_path(tema, folder)
            unzip(fname, tema_folder)
        else:
            print(f'Dados para tema {tema} já disponíveis')
    #preciso mandar os shapes pra fgv por email    
    #delete_existing_files(folder, extension='.zip')

In [9]:
de_para_arquivos = {
    'regiao_5' : '01_Limites%20Administrativos%5C%5CRegi%E3o%205%5C%5CShapefile%5C%5CREGIAO5',
    'distritos' : '01_Limites Administrativos\\Distrito\\Shapefile\\SIRGAS_SHP_distrito',
    'subprefeituras' : '01_Limites Administrativos\\Subprefeituras\\Shapefile\\SIRGAS_SHP_subprefeitura'
    
}

In [10]:
download_all_files(de_para_arquivos)

Dados para tema regiao_5 já disponíveis
Dados para tema distritos já disponíveis
Dados para tema subprefeituras já disponíveis


In [11]:
def filtrar_sirgas(tema):
    
    shps = find_shps(tema)
    
    return [shp for shp in shps if 'SIRGAS' in shp][0]

In [12]:
regiao_5 = gpd.read_file(filtrar_sirgas('regiao_5'))

In [13]:
subs = gpd.read_file(filtrar_sirgas('subprefeituras'))

In [14]:
geometria_subs = subs['geometry'].copy()
subs['geometry'] = subs['geometry'].centroid

In [15]:
subs = gpd.sjoin(subs, regiao_5, how='left', predicate='within')

In [16]:
subs['geometry'] = geometria_subs

In [17]:
distritos = gpd.read_file(filtrar_sirgas('distritos'))

In [18]:
set(subs['sp_codigo']) - set(distritos['ds_cd_sub'])

set()

In [19]:
set(distritos['ds_cd_sub'])- set(subs['sp_codigo']) 

set()

In [20]:
distritos.rename({'ds_cd_sub' : 'cod_subprefeitura'}, axis =1, inplace=True)

In [21]:
distritos.keys()

Index(['ds_areamt', 'ds_subpref', 'ds_sigla', 'ds_nome', 'ds_areakm',
       'cod_subprefeitura', 'ds_codigo', 'geometry'],
      dtype='object')

In [22]:
distritos = distritos[['ds_codigo', 'ds_sigla', 'ds_nome', 'cod_subprefeitura']]

In [23]:
subs = subs[[
    'sp_codigo',
    'sp_nome',
    'COD_REG_5'
]]

In [24]:
subs = subs.rename({'COD_REG_5' : 'cod_reg_5',
                   'sp_codigo' : 'cod_subprefeitura'}, axis=1)

In [25]:
regiao_5.rename({'COD_REG_5' : 'cod_reg_5',
                'NOME' : 'nome'}, axis =1, inplace=True)

In [26]:
regiao_5 = regiao_5[['cod_reg_5', 'nome']]

In [27]:
def save_df(df, fname, folder):
    
    fname = solve_path(fname, folder)
    
    try:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='latin-1', index=False)
    except UnicodeEncodeError:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='utf-8', index=False)

In [28]:
save_df(regiao_5, 'regiao_5.csv', GENERATED_DATA_DIR)

In [29]:
save_df(subs, 'subprefeituras.csv', GENERATED_DATA_DIR)

In [30]:
save_df(distritos, 'distritos.csv', GENERATED_DATA_DIR)